In [0]:
				df = spark.readStream.format("kafka") \
				.option("kafka.bootstrap.servers" , "98.81.131.195:9092") \
				.option("subscribe", "ticker_quotes")\
				.option("startingOffsets" , "earliest") \
				.option("failOnDataLoss" , "false")\
.load()

In [0]:
df.printSchema()


In [0]:
ACCESS_KEY = dbutils.secrets.get(scope = "ticker", key = "access_key")
SECRET_KEY = dbutils.secrets.get(scope = "ticker", key = "secret_key")
SESSION_TOKEN = dbutils.secrets.get(scope = "ticker", key = "session_key")

Trigger type ProcessingTime is not supported for this cluster type.
Use a different trigger type e.g. AvailableNow, Once. SQLSTATE: 0A000
You are in a very restrictive environment (Shared Access Mode on Databricks Community Edition). This mode has three major blocks:

Global Configs Blocked: You can't set global Hadoop keys.

Mounts Blocked: You can't mount buckets.

Root DBFS Blocked: You can't write to /dbfs/.

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.default.storage;

In [0]:
%sql
SHOW VOLUMES IN workspace.default

In [0]:
%fs mkdirs /Volumes/workspace/default/storage/checkpoints

In [0]:


BUCKET = "mzon-to-databricks-5482"
destination_path = "/Volumes/workspace/default/storage/bronze/ticker_data"
# --- 2. THE FIX: PATH MUST INCLUDE THE VOLUME NAME ---
# Path format: /Volumes/<catalog>/<schema>/<VOLUME_NAME>/<folder>
# We added 'storage' because that is the volume we just created in SQL.
checkpoint_path = "/Volumes/workspace/default/storage/checkpoints/job_bronze_ticker"

print(f"Streaming Strategy:")
print(f"Checkpoint: {checkpoint_path}")
print(f"Data (S3): {destination_path}")

# --- 3. WRITE STREAM ---
df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_path) \
    .option("fs.s3a.access.key", ACCESS_KEY) \
    .option("fs.s3a.secret.key", SECRET_KEY) \
    .option("fs.s3a.session.token", SESSION_TOKEN) \
    .option("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider") \
    .trigger(availableNow=True) \
    .start(destination_path)

In [0]:
# df.writeStream \
# 				 .format("delta") \
# 				 .outputMode("append") \
# .option("checkpointLocation" , f"file:/tmp/checkpoints/job_bronze_ticker") \
#             .option("fs.s3a.access.key", ACCESS_KEY) \
#         .option("fs.s3a.secret.key", SECRET_KEY) \
#         .option("fs.s3a.session.token", SESSION_TOKEN) \
#                 .option("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider") \
#         .trigger(availableNow=True) \
#         .start(destination_path)

In [0]:
# dbutils.fs.rm(checkpoint_path, True)
# dbutils.fs.rm(destination_path, True)

In [0]:
# %fs rm -r /Volumes/workspace/default/storage/checkpoints/job_bronze_ticker

In [0]:
# %fs rm -r /Volumes/workspace/default/storage/bronze/ticker_data

In [0]:
# DIAGNOSTIC TOOL: Check Kafka Content
df_debug = spark.read.format("kafka") \
    .option("kafka.bootstrap.servers", "98.81.131.195:9092") \
    .option("subscribe", "ticker_quotes") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

count = df_debug.count()

print(f"--- DIAGNOSIS REPORT ---")
print(f"Total records found in Kafka Topic: {count}")

if count > 0:
    print("GOOD NEWS: Data exists! The issue is your Streaming Checkpoint.")
    display(df_debug.limit(5))
else:
    print("BAD NEWS: The Kafka Topic is empty. You need to restart your Python Producer.")